In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys

from tqdm import notebook
from fake_useragent import UserAgent

In [2]:
main_link = 'https://www.labirint.ru/books/?page=' 
page_link = 'https://www.labirint.ru'

In [3]:
def get_soup(page_link):
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [4]:
def get_product_links(page_num):
    try:
        soup = get_soup(main_link + str(page_num))
        books_div = soup.findAll("div", attrs={'class':'products-row', 'data-title':'Все в жанре «Книги»'})    
        products = books_div[0].findAll("div", attrs={'class':'product-cover'})
        product_href = [product.find('a', attrs={'class':'cover'})['href'] for product in products]
    except:
        product_href = None
    return product_href

In [5]:
def _get_id(product_soup):
    _id = product_soup.find("div", attrs={'class':'articul'})    
    if not _id:
        return None
    _id = re.findall('\d+', _id.text )
    if not len(_id):
        return 0
    return _id[0]     
    
def _get_title(product_soup):
    title = product_soup.find("h1")        
    title = None if not title else title.text[title.text.find(':')+1:].strip()
    return title

def _get_author(product_soup):
    try:
        authors = product_soup.find("div", attrs={'class':'authors'}).findAll("a")
        authors = [author.text.strip() for author in authors]
    except:
        authors = None
    return authors

def _get_rating(product_soup):
    rating = product_soup.find("div", attrs={'id':'rate'})
    rating = None if not rating else rating.text.strip()
    return rating

def _get_marks_cnt(product_soup):
    # Оценить (оценило: 16) - пример содержимого div-а
    marks_cnt = product_soup.find("div", attrs={'id':'product-rating-marks-label'})    
    if not marks_cnt:
        return None
    marks_cnt = re.findall('\d+', marks_cnt.text )
    if not len(marks_cnt):
        return 0
    return marks_cnt[0]    

def _get_publisher(product_soup):
    try:
        publisher = product_soup.find('div', attrs={'class':'publisher'}).find('a')
        publisher = publisher.text.strip()
    except:
        publisher = None
    return publisher

def _get_series(product_soup):
    try:
        series = product_soup.find('div', attrs={'class':'series'}).find('a')
        series = series.text.strip()
    except:
        series = None
    return series

def _get_publication_year(product_soup):
    publication_year = product_soup.find('div', attrs={'class':'publisher'})
    publication_year = None if not publication_year else publication_year.text[-7:-3].strip()    
    return publication_year

def _get_genres(product_soup):
    try:
        genres = product_soup.find('div', attrs={'id':'thermometer-books'}).findAll('span', attrs={'class':'thermo-item'})
        genres = None if not genres else [genre.text[:-2].strip() for genre in genres]
    except:
        genres = None
    return genres

def _get_isbn(product_soup):
    isbn = product_soup.find("div", attrs={'class':'isbn'})
    isbn = None if not isbn else isbn.text[5:].strip()
    return isbn

def _get_annotation(product_soup):
    annotation = product_soup.find('div', attrs={'id':'fullannotation'})
    if not annotation:
        try:
            annotation = product_soup.find('div', attrs={'id':'product-about'}).find('p')        
        except:
            annotation = None
    annotation = None if not annotation else annotation.text.strip()
    return annotation
    
def _get_price(product_soup):
    price = product_soup.find("span", attrs={'class':'buying-priceold-val-number'})
    if not price:
        price = product_soup.find("span", attrs={'class':'buying-price-val-number'})
    price = None if not price else price.text.strip()
    return price

def get_product_page_info(product_link):    
    product_soup = get_soup(page_link + product_link)
    _id = _get_id(product_soup)
    authors = _get_author(product_soup)
    title = _get_title(product_soup)
    publisher  = _get_publisher(product_soup)
    publication_year = _get_publication_year(product_soup)
    series = _get_series(product_soup)
    isbn = _get_isbn(product_soup)
    genres = _get_genres(product_soup)
    rating = _get_rating(product_soup)
    marks_cnt = _get_marks_cnt(product_soup)
    price = _get_price(product_soup)
    annotation = _get_annotation(product_soup) 
    
    info = {
        'id': _id,
        'authors': authors,
        'title':title,
        'publisher': publisher,
        'publication_year':publication_year,
        'series':series,
        'isbn': isbn,
        'genres': genres,
        'rating': rating,
        'marks_cnt': marks_cnt,
        'price':price,
        'annotation': annotation
    }
    
    return info

In [6]:
final_df = pd.DataFrame(columns=['id', 'authors', 'title', 'publisher', 'publication_year',
                                 'series', 'isbn', 'genres', 'rating',
                                 'marks_cnt','price', 'annotation'])

In [7]:
links = get_product_links(20)

In [8]:
for book_link in notebook.tqdm(links):
    data_row = get_product_page_info(book_link)
    final_df = final_df.append(data_row, ignore_index=True)

In [9]:
final_df

,id,authors,title,publisher,publication_year,series,isbn,genres,rating,marks_cnt,price,annotation
0,745956,[Жакорт Фиби],Гигантозавр. История появления,Редакция Вилли Винки,2020,Гигантозавр,978-5-17-120670-3,"[Книги для детей, Детская художественная литер...",0.00,0,576,"Мазу, Роки, Билл и Кроха - четверо маленьких д..."
1,746128,None,"Улётные приключения Миши и Сашки из 2""Б""",Малыш,2020,Школьные истории,978-5-17-122316-8,"[Книги для детей, Детская художественная литер...",10.00,6,482,Как-то на уроке рисования Сашка изобразил обык...
2,747232,None,Pasternak,АСТ,2020,None,978-5-17-122122-5,"[Художественная литература, Современная проза,...",10.00,1,644,"Михаил Елизаров - автор романов ""Земля"", ""Библ..."
3,748823,[Балуева Оксана],Мир животных,Проф-Пресс,2020,Книжки на картоне ЦК мини,978-5-378-29813-6,"[Книги для детей, Первые книги малыша. Развити...",10.00,1,93,Книги данной серии предназначены для самых мал...
4,748865,None,1000 первых знаний в картинках,АСТ,2020,Большая книга малыша: от 6 месяцев до 3 лет,978-5-17-122549-0,"[Книги для детей, Первые книги малыша. Развити...",0.00,0,420,"Серия ""Большая книга малыша"" поможет ребёнку з..."
5,748866,[Толстой Алексей Николаевич],"Приключения Буратино, или Золотой Ключик",АСТ,2020,Главные книги для детей,978-5-17-118720-0,"[Книги для детей, Детская художественная литер...",9.00,1,482,Алексей Николаевич Толстой очень любил в детст...
6,748870,[Сандрель Жюльен],Комната чудес,Corpus,2020,roman,978-5-17-109417-1,"[Художественная литература, Современная проза,...",0.00,0,601,"В жизни Тельмы, успешного директора по маркети..."
7,748871,[Баркс Карл],Дональд Дак. Ужасные Братья Гавс,АСТ,2020,Disney Comics. Утиные истории,978-5-17-121138-7,"[Книги для детей, Детская художественная литер...",10.00,2,1011,Первое противостояние дядюшки Скруджа и Братье...
8,748873,[Трофимов Ерофей],Казачий спас,АСТ,2020,Боевая фантастика,978-5-17-121478-4,"[Художественная литература, Фантастика, Отечес...",0.00,0,368,Многое рассказывают о них. Что и воины они вел...
9,748874,[Блиш Джеймс],Поверхностное натяжение,АСТ,2020,Мастера фантазии,978-5-17-115632-9,"[Художественная литература, Фантастика, Зарубе...",0.00,0,870,"Джеймс Блиш (1921-1975) - представитель ""золот..."
